In [1]:
from IPython.display import clear_output
import re

## Get Candidates

In [2]:
import datetime
import time
from tqdm import tqdm_notebook as tqdm
from github import Github
g = Github("dkbrz", "")

In [39]:
def dates():
    d1 = datetime.date(2007, 10, 28)  # start date
    d2 = datetime.date(2018, 11, 13)  # end date
    delta = d2 - d1         # timedelta
    for i in range(delta.days + 1):
        yield str(d1 + datetime.timedelta(i))

In [5]:
def get_dates():
    q = dates()
    stop = False
    while not stop:
        a = []
        for i in range(10):
            try:
                a.append(next(q))
            except StopIteration:
                stop = True
        if len(a) > 0:
            yield a

In [38]:
def _write_name(chunk, f):
            date = ' '.join('created:{}'.format(date) for date in chunk)
            S = g.search_repositories('stars:>200 {}'.format(date), sort='stars', order='desc')
            S._PaginatedList__requester.per_page = 100
            for repo in S:
                f.write(repo.full_name+'\n')
            print (chunk[-1])
            time.sleep(2)
            clear_output()

In [32]:
def get_repos():
    with open('repos.txt','a', encoding='utf-8') as f:
        for chunk in get_dates():
            try:
            #if True:
                _write_name(chunk, f)
            except:
                print ('Oops!')
                print (chunk)
                time.sleep(5)
                _write_name(chunk, f)

In [40]:
get_repos()

In [41]:
k = 0
with open('repos.txt','r') as f:
    for line in f:
        k += 1
k

74726

## Get Commits

In [4]:
import sqlite3
con = sqlite3.connect('DB.db')
cur = con.cursor()

In [5]:
def _next_repo():
    with open('repos.txt','r') as f:
        for line in f: 
            if line != '\n': yield line.strip()

In [6]:
def _parse_commit(commit, R):
    try:
        author = re.sub('-{2,}','\\-\\-',commit.author.login).replace('"','""')
    except: author = ''
    message = re.sub('-{2,}','\\-\\-',commit.commit.message).replace('"','""')
    gh_id = R.id
    date = commit.commit.author.date
    result = (author, gh_id, message, date)
    return result

In [36]:
def _chunks_of_commits(C, R):
    stop = False
    while not stop:
        a = []
        for _ in range(1000):
            try:
                k = next(C)
                a.append(_parse_commit(k, R))
            except: break
        if len(a) > 0: yield a
        else: raise StopIteration

In [15]:
def _save_repo(R):
    global cur
    cur.execute('\
    INSERT INTO R (gh_id, owner, name, language, created_at, updated_at,\
    size, stars, subs, watchers, network_count, has_wiki, languages_url)\
    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)',
               (R.id, R.owner.login, R.name, R.language, R.created_at, R.updated_at,
                R.size, R.stargazers_count, R.subscribers_count, R.watchers_count,
                R.network_count, int(R.has_wiki), R.languages_url)
               )

In [16]:
def _write_data(R, C, nPages):
    global cur, con
    for chunk in tqdm(_chunks_of_commits(C, R)):
        cur.executemany('INSERT INTO C (user, repo_id, text, date) VALUES (?, ?, ?, ?)', chunk)
    _save_repo(R)
    con.commit()

In [39]:
def get_data():
    with open('later.txt', 'a') as f:
        for full_name in _next_repo():
            try:
                print(full_name)
                R = g.get_repo(full_name)
                if R.fork: break
                C = R.get_commits()
                C._PaginatedList__requester.per_page = 100
                nPages = C.totalCount // 100 + 1
                print (nPages/10)
                if nPages >  4000:
                    f.write('{}\t{}\n'.format(full_name, nPages))
                    break
                else:
                    left = g.get_rate_limit().core
                    nLeft = left.remaining
                    timing = (datetime.datetime.utcnow() - left.reset).seconds
                    if nPages >= nLeft + 5:
                        if timing > 300: f.write('{}\t{}\n'.format(full_name, nPages))
                        else: 
                            print (left.reset)
                            time.sleep(timing)
                    else:
                        C = iter(C)
                        _write_data(R, C, nPages)
                clear_output()
            except:
                print ('Oops!')
                f.write('{}\t{}\n'.format(full_name, -1))
                        

In [40]:
con.rollback()

In [ ]:
get_data()

git/git
53.9


In [42]:
g.get_rate_limit().core

Rate(remaining=5000, limit=5000)

In [15]:
R = g.get_repo('mojombo/grit')

In [15]:
str(R.created_at)

'2011-07-29 21:19:00'

In [121]:
R.language

'CSS'

In [46]:
C = R.get_commits()

In [47]:
c = C[0]

In [53]:
c.author.login

'bkeepers'

In [26]:
E = iter(C)

In [41]:
next(E)

Commit(sha="f87da75a4ab2f2de7201370c9e8b1da99e703eef")

In [76]:
C1 = C[0]

In [77]:
C1

Commit(sha="0a3755b30ade6137dc3d51238e729f741ec5983a")

In [84]:
C1.commit.message

'Fix typo in the Popper.js checks. (#27488)'

In [87]:
C1.commit.author.name

'XhmikosR'

In [88]:
C1.commit.author.date

datetime.datetime(2018, 10, 21, 8, 51, 20)

In [83]:
C1.stats.total

4